# Build and Deploy model using seldon-core

## Build Your Model

In [4]:
import sklearn
import pandas as pd
import joblib
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import os

In [5]:
boston_data = load_boston()

In [6]:
df = pd.DataFrame(boston_data.data, columns = boston_data.feature_names)

In [7]:
target = boston_data.target

In [8]:
rf_model = RandomForestRegressor(n_jobs=-1, max_depth=5, random_state=42)

In [9]:
xtrain, xtest, ytrain, ytest = train_test_split(df, target, test_size=0.2, random_state=42)

In [10]:
rf_model.fit(xtrain, ytrain)

RandomForestRegressor(max_depth=5, n_jobs=-1, random_state=42)

In [11]:
r2_score(ytest, rf_model.predict(xtest))

0.8785132418642353

In [12]:
if not os.path.exists('app/boston_rf_model'):
    os.makedirs('app/boston_rf_model')

In [18]:
with open('app/boston_rf_model/model.jlib', "wb") as mf:
    joblib.dump(rf_model, mf)

In [19]:
%%writefile app/BostonModel.py
import joblib
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("BOSTON APP")
class BostonModel:
    def __init__(self):
        logger.info("Loading Model")
        self.model =  joblib.load('boston_rf_model/model.jlib')
    
    def predict(self, *args, **kwargs):
        logger.info(args)
        logger.info(kwargs)
        data = args[0]
        logger.info(data)
        output = self.model.predict(data).tolist()
        logger.info(output)
        return output


Overwriting app/BostonModel.py


## Test APP

In [20]:
from app.BostonModel import BostonModel
model = BostonModel()
predictions = model.predict([[0.00632, 18.0, 2.31, 0.0, 0.538, 6.575, 65.2, 4.09, 1.0, 296.0, 15.3, 396.9, 4.98], 
               [0.02731, 0.0, 7.07, 0.0, 0.469, 6.421, 78.9, 4.9671, 2.0, 242.0, 17.8, 396.9, 9.14], 
               [0.02729, 0.0, 7.07, 0.0, 0.469, 7.185, 61.1, 4.9671, 2.0, 242.0, 17.8, 392.83, 4.03], 
               [0.03237, 0.0, 2.18, 0.0, 0.458, 6.998, 45.8, 6.0622, 3.0, 222.0, 18.7, 394.63, 2.94], 
               [0.06905, 0.0, 2.18, 0.0, 0.458, 7.147, 54.2, 6.0622, 3.0, 222.0, 18.7, 396.9, 5.33]])
assert len(predictions) == 5 and sum(predictions) > 0

INFO:BOSTON APP:Loading Model
INFO:BOSTON APP:([[0.00632, 18.0, 2.31, 0.0, 0.538, 6.575, 65.2, 4.09, 1.0, 296.0, 15.3, 396.9, 4.98], [0.02731, 0.0, 7.07, 0.0, 0.469, 6.421, 78.9, 4.9671, 2.0, 242.0, 17.8, 396.9, 9.14], [0.02729, 0.0, 7.07, 0.0, 0.469, 7.185, 61.1, 4.9671, 2.0, 242.0, 17.8, 392.83, 4.03], [0.03237, 0.0, 2.18, 0.0, 0.458, 6.998, 45.8, 6.0622, 3.0, 222.0, 18.7, 394.63, 2.94], [0.06905, 0.0, 2.18, 0.0, 0.458, 7.147, 54.2, 6.0622, 3.0, 222.0, 18.7, 396.9, 5.33]],)
INFO:BOSTON APP:{}
INFO:BOSTON APP:[[0.00632, 18.0, 2.31, 0.0, 0.538, 6.575, 65.2, 4.09, 1.0, 296.0, 15.3, 396.9, 4.98], [0.02731, 0.0, 7.07, 0.0, 0.469, 6.421, 78.9, 4.9671, 2.0, 242.0, 17.8, 396.9, 9.14], [0.02729, 0.0, 7.07, 0.0, 0.469, 7.185, 61.1, 4.9671, 2.0, 242.0, 17.8, 392.83, 4.03], [0.03237, 0.0, 2.18, 0.0, 0.458, 6.998, 45.8, 6.0622, 3.0, 222.0, 18.7, 394.63, 2.94], [0.06905, 0.0, 2.18, 0.0, 0.458, 7.147, 54.2, 6.0622, 3.0, 222.0, 18.7, 396.9, 5.33]]
INFO:BOSTON APP:[27.810927390868514, 22.462326702578

In [21]:
%%writefile app/requirements.txt
seldon-core==1.4.0
scikit-learn==0.23.1
joblib


Overwriting app/requirements.txt


## Build Docker Image

In [17]:
%%writefile Dockerfile
FROM python:3.7-slim
COPY app /app
WORKDIR /app
RUN pip install -r requirements.txt
EXPOSE 5000

# Define environment variable
ENV MODEL_NAME BostonModel
ENV SERVICE_TYPE MODEL
ENV PERSISTENCE 0
ENV API_TYPE REST

CMD exec seldon-core-microservice $MODEL_NAME $API_TYPE --service-type $SERVICE_TYPE --persistence $PERSISTENCE



Overwriting Dockerfile


```shell
docker build . -t maytest/boston_mode:v1 --no-cache
```

## Run Docker Image

```shell
docker run --rm --name mymodel -p 5000:5000 maytest/boston_mode:v1
```

# Predcit

```shell
curl -X POST -H 'Content-Type: application/json' -H 'Accept: application/json' -d '{"data":{"ndarray":[[0.00632, 18.0, 2.31, 0.0, 0.538, 6.575, 65.2, 4.09, 1.0, 296.0, 15.3, 396.9, 4.98]]}}' http://localhost:5000/api/v1.0/predictions
```